In [1]:
import pandas as pd
import numpy as np

In [2]:
ap_data = pd.read_excel("all_data_model.xlsx")

AttributeError: 'DataFrame' object has no attribute 'legnth'

In [3]:
ap_data.set_index(ap_data['date'], inplace=True)

In [4]:
ap_data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 672 entries, 2019-09-20 00:00:00 to 2019-10-17 23:00:00
Data columns (total 18 columns):
date              672 non-null datetime64[ns]
all_num           672 non-null int64
temp              672 non-null float64
humid             672 non-null float64
co2               672 non-null float64
voc               672 non-null float64
pm25              672 non-null float64
light             672 non-null float64
noise             672 non-null float64
기온                672 non-null float64
강수량               672 non-null float64
풍속                672 non-null float64
습도                672 non-null int64
date_day          672 non-null int64
date_hour         672 non-null int64
date_weekday      672 non-null int64
model_number_1    672 non-null float64
model_number_2    672 non-null float64
dtypes: datetime64[ns](1), float64(12), int64(5)
memory usage: 99.8 KB


In [5]:
import matplotlib.pyplot as plt

plt.figure(figsize = (20,15))
xticks = pd.date_range(start = '2019-09-20', end ='2019-10-17')
week  = ["일","월","화","수","목","금","토"]
xticks_name = [week[int(w)]+d for w, d in zip(xticks.strftime("%w"),
                                             xticks.strftime(" %m-%d"))]
plt.xticks(xticks,xticks_name , rotation = 90, ha = "left")
plt.plot(ap_data['model_number_2'],linewidth = 1)
plt.xlabel("날짜")
plt.ylabel("대여횟수")

C:\Users\user\Anaconda3\envs\tensorflow\lib\site-packages\pandas\plotting\_matplotlib\converter.py:102: FutureWarning: Using an implicitly registered datetime converter for a matplotlib plotting method. The converter was registered by pandas on import. Future versions of pandas will require you to explicitly register matplotlib converters.

To register the converters:
	>>> from pandas.plotting import register_matplotlib_converters
	>>> register_matplotlib_converters()
  warnings.warn(msg, FutureWarning)


Text(0, 0.5, '대여횟수')

C:\Users\user\Anaconda3\envs\tensorflow\lib\site-packages\matplotlib\backends\backend_agg.py:211: RuntimeWarning: Glyph 44552 missing from current font.
  font.set_text(s, 0.0, flags=flags)
C:\Users\user\Anaconda3\envs\tensorflow\lib\site-packages\matplotlib\backends\backend_agg.py:211: RuntimeWarning: Glyph 53664 missing from current font.
  font.set_text(s, 0.0, flags=flags)
C:\Users\user\Anaconda3\envs\tensorflow\lib\site-packages\matplotlib\backends\backend_agg.py:211: RuntimeWarning: Glyph 51068 missing from current font.
  font.set_text(s, 0.0, flags=flags)
C:\Users\user\Anaconda3\envs\tensorflow\lib\site-packages\matplotlib\backends\backend_agg.py:211: RuntimeWarning: Glyph 50900 missing from current font.
  font.set_text(s, 0.0, flags=flags)
C:\Users\user\Anaconda3\envs\tensorflow\lib\site-packages\matplotlib\backends\backend_agg.py:211: RuntimeWarning: Glyph 54868 missing from current font.
  font.set_text(s, 0.0, flags=flags)
C:\Users\user\Anaconda3\envs\tensorflow\lib\site-p

In [6]:
from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, LassoLarsIC
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor
import xgboost as xgb
import lightgbm as lgb
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
from sklearn.model_selection import GridSearchCV

In [8]:
ntrain = 450
train = ap_data[:ntrain]
test = ap_data[ntrain:]
y_train = ap_data['model_number_2'].values

In [22]:
y_train

array([  1.7787914,   0.       ,   0.       ,   0.       ,   0.       ,
         0.       ,   0.       ,   7.1151656,  17.787914 ,  32.0182452,
        44.469785 ,  49.8061592,  39.1334108,  51.5849506,  53.363742 ,
        72.9304474,  56.9213248,  51.5849506,  39.1334108,  30.2394538,
        21.3454968,  28.4606624,  24.9030796,  10.6727484,   3.5575828,
         0.       ,   0.       ,   0.       ,   0.       ,   0.       ,
         0.       ,   8.893957 ,  10.6727484,  17.787914 ,  19.5667054,
        23.1242882,  24.9030796,  26.681871 ,  46.2485764,  53.363742 ,
        62.257699 ,  39.1334108,  37.3546194,  23.1242882,  28.4606624,
        24.9030796,  14.2303312,  12.4515398,   0.       ,   0.       ,
         0.       ,   0.       ,   0.       ,   0.       ,   1.7787914,
         1.7787914,   1.7787914,   3.5575828,  16.0091226,  26.681871 ,
        30.2394538,  28.4606624,  16.0091226,  40.9122022,  28.4606624,
        23.1242882,  30.2394538,  23.1242882,  28.4606624,  32.0

In [9]:
#Validation function
n_folds = 10

def rmsle_cv(model):
    kf = KFold(n_folds, shuffle=True, random_state=42).get_n_splits(train.values)
    rmse= np.sqrt(-cross_val_score(model, train.values, y_train, scoring="neg_mean_squared_error", cv = kf))
    return(rmse)

In [10]:
GBoost = GradientBoostingRegressor(n_estimators=3000, learning_rate=0.05,
                                   max_depth=4, max_features='sqrt',
                                   min_samples_leaf=15, min_samples_split=10, 
                                   loss='huber', random_state =5)

In [11]:
model_xgb = xgb.XGBRegressor(colsample_bytree=0.4603, gamma=0.0468, 
                             learning_rate=0.05, max_depth=3, 
                             min_child_weight=1.7817, n_estimators=2200,
                             reg_alpha=0.4640, reg_lambda=0.8571,
                             subsample=0.5213, silent=1,
                             random_state =7, nthread = -1)

In [12]:
model_lgb = lgb.LGBMRegressor(objective='regression',num_leaves=5,
                              learning_rate=0.05, n_estimators=720,
                              max_bin = 55, bagging_fraction = 0.8,
                              bagging_freq = 5, feature_fraction = 0.2319,
                              feature_fraction_seed=9, bagging_seed=9,
                              min_data_in_leaf =6, min_sum_hessian_in_leaf = 11)

In [13]:
model_svr = SVR(C=1, cache_size=200, coef0=0, degree=3, epsilon=0.0, gamma='auto',
  kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)


In [14]:
#params = {'gamma' :[i/100.0 for i in range(0,11)],  
#          'coef0':[0, 0.1, 0.5, 1], 'C' :[0.1, 0.2, 0.5, 1], 'epsilon':[i/10.0 for i in range(0,6)]}

params = {'coef0':[0, 0.1, 0.5, 1], 'C' :[0.1, 0.2, 0.5, 1], 'epsilon':[i/10.0 for i in range(0,6)]}


#model_svr = SVR()
#grid_search = GridSearchCV(model_svr, params, cv=10, scoring='neg_mean_squared_error')
#grid_search.fit(train, y_train)

In [15]:
regr = RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features=150, max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=90, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [16]:
score = rmsle_cv(GBoost)
print("Gradient Boosting score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

ValueError: Found input variables with inconsistent numbers of samples: [450, 672]

In [21]:
score = rmsle_cv(model_xgb)
print("Xgboost score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

ValueError: Found input variables with inconsistent numbers of samples: [450, 672]

In [ ]:
score = rmsle_cv(model_lgb)
print("LGBM score: {:.4f} ({:.4f})\n" .format(score.mean(), score.std()))

In [ ]:
score = rmsle_cv(regr)
print("RandomForestReg score: {:.4f} ({:.4f})\n" .format(score.mean(), score.std()))

In [ ]:
#define a rmsle evaluation function

def rmsle(y, y_pred): 
    return np.sqrt(mean_squared_error(y, y_pred))

In [17]:
model_xgb.fit(train, y_train)
xgb_train_pred = model_xgb.predict(train)
xgb_pred = np.expm1(model_xgb.predict(test))
print(rmsle(y_train, xgb_train_pred))

ValueError: DataFrame.dtypes for data must be int, float or bool.
                Did not expect the data types in fields date

In [18]:
model_lgb.fit(train, y_train)
lgb_train_pred = model_lgb.predict(train)
lgb_pred = np.expm1(model_lgb.predict(test.values))
print(rmsle(y_train, lgb_train_pred))

ValueError: DataFrame.dtypes for data must be int, float or bool.
Did not expect the data types in the following fields: date

In [19]:
GBoost.fit(train,y_train)
GB_train_pred = GBoost.predict(train)
GB_pred = np.expm1(GBoost.predict(test.values))
print(rmsle(y_train, GB_train_pred))

TypeError: float() argument must be a string or a number, not 'Timestamp'